In [2]:
from tqdm.autonotebook import tqdm
import xarray as xr
import os
import pandas as pd
import numpy as np
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 
from get_era5_around_tgs import get_era5_around_tgs

/tmp/ipykernel_115623/1980157682.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Configure the script:

In [4]:
grid_size_around_tgs=2 #degrees around TGs

predictand_path = '/home/jovyan/test_surge_models/input/t_tide_6h_hourly_deseasoned_predictands'
predictor_path = 'leap-persistent/timh37/era5/6hourly'

tg_coords = xr.open_dataset('input/gesla3_tg_coordinates_eu.nc')#.sel(tg=['den_helder-denhdr-nld-rws.csv']) #load TG coordinates

In [6]:
#open predictors
da = xr.open_mfdataset(['gs://'+x for x in fs.ls(predictor_path) if '.nc' in x],engine='zarr',concat_dim="time", combine="nested",
                      data_vars='minimal', coords='minimal', compat='override').chunk({'time':100000})#,chunks={'longitude':30,'latitude':30}).chunk({'time':100000})
da.attrs['resolution'] = '0p25x0p25'

for output_tg in ['barcelona-bar-esp-cmems.csv','den_helder-denhdr-nld-rws.csv', 'brest-822a-fra-uhslc.csv', 
            'immingham-imm-gbr-bodc.csv','stavanger-svg-nor-nhs.csv']:#tg_coords.tg.values:
    print('processing: '+output_tg)
    da_around_tgs = get_era5_around_tgs(da,grid_size_around_tgs,tg_coords.sel(tg=[output_tg])).chunk({'tg':1}) #get predictors around each TG


    output_fn = 'era5_'+str(grid_size_around_tgs)+'x'+str(grid_size_around_tgs)+'_around_'+ output_tg.replace('.csv','.zarr')
    
    #if output_tg.replace('.csv','_era5Predictors_'+str(grid_size_around_tgs)+'x'+str(grid_size_around_tgs)+'.nc') not in os.listdir('predictors/'):
        #da_around_tgs.sel(tg=output_tg).to_netcdf('predictors/'+output_tg.replace('.csv','_era5Predictors_'+str(grid_size_around_tgs)+'x'+str(grid_size_around_tgs)+'.nc'))
    #da_around_tgs.sel(tg=output_tg).to_zarr(os.path.join('gs://leap-persistent/timh37/era5/6hourly/5x5_around_gesla3',output_fn),mode='w')
    da_around_tgs.sel(tg=output_tg).to_netcdf(output_fn.replace('.zarr','.nc'),mode='w')

processing: barcelona-bar-esp-cmems.csv
processing: den_helder-denhdr-nld-rws.csv
processing: brest-822a-fra-uhslc.csv
processing: immingham-imm-gbr-bodc.csv
processing: stavanger-svg-nor-nhs.csv


In [10]:
os.path.join('gs://leap-persistent/timh37/era5/6hourly/5x5_around_gesla3',output_fn)

'gs://leap-persistent/timh37/era5/6hourly/5x5_around_gesla3/era5_5x5_around_aberdeen-abe-gbr-bodc.nc'